In [1]:
"""
!pip install convokit
!pip install transformers[torch]
!pip install accelerate
# !pip install transformers_interpret
!pip install git+https://github.com/allenai/longformer.git
!pip install jsonlines
!pip install shap
!pip install openai==1.12.0     # Azure AI
!pip install openai==0.28       # OPENAI API
"""

  Cloning https://github.com/allenai/longformer.git to c:\users\17245\appdata\local\temp\pip-req-build-72gi44uy
  Resolved https://github.com/allenai/longformer.git to commit caefee668e39cacdece7dd603a0bebf24df6d8ca
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Cloning http://github.com/ibeltagy/transformers.git (to revision longformer_encoder_decoder) to c:\users\17245\appdata\local\temp\pip-install-g6jeb9si\transformers_9890f9da687c4a98ab06ffe32bb4da31
  Resolved http://github.com/ibeltagy/transformers.git to commit 52d6236dc15ad5142b4146ff74d2ec973fa3da22
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Cloning http://github.com/ibeltagy/pytorch-lightning.git (to revision v0.8.5_fixes) to c:\users\17245\appdata\local\temp\pip-install-g6jeb9si\pytorch-lightning_3bb7c3f9e9164654a3cc2f2b04dff914
  Resolved http://github.com/ibeltagy/pytorch-lightning.git to commit 7ed5d849a0

  Running command git clone --filter=blob:none --quiet https://github.com/allenai/longformer.git 'C:\Users\17245\AppData\Local\Temp\pip-req-build-72gi44uy'
  Running command git clone --filter=blob:none --quiet http://github.com/ibeltagy/transformers.git 'C:\Users\17245\AppData\Local\Temp\pip-install-g6jeb9si\transformers_9890f9da687c4a98ab06ffe32bb4da31'
  Running command git checkout -b longformer_encoder_decoder --track origin/longformer_encoder_decoder
  branch 'longformer_encoder_decoder' set up to track 'origin/longformer_encoder_decoder'.
  Switched to a new branch 'longformer_encoder_decoder'
  Running command git clone --filter=blob:none --quiet http://github.com/ibeltagy/pytorch-lightning.git 'C:\Users\17245\AppData\Local\Temp\pip-install-g6jeb9si\pytorch-lightning_3bb7c3f9e9164654a3cc2f2b04dff914'
  Running command git checkout -b v0.8.5_fixes --track origin/v0.8.5_fixes
  branch 'v0.8.5_fixes' set up to track 'origin/v0.8.5_fixes'.
  Switched to a new branch 'v0.8.5_fixes'


Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import matplotlib.pyplot as plt
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
import time
from sklearn import tree
import pandas as pd
import jsonlines
from datetime import date
import csv
import openai
openai.api_key = "XXXXXXXXXXX"

import json
import random
import torch
from torch.nn import L1Loss
from sklearn.metrics import mean_absolute_error
import numpy as np
from sklearn.metrics import mean_squared_error
from scipy import stats

import convokit
import spacy
from convokit import Corpus, Speaker, Utterance
from convokit import download
from convokit import TextParser



# 0. Load Data Function

## 0.1. OUM data

In [3]:
# Load the data
def load_data_oum(label='after'):
    final_convs = []
    final_labels = []
    wizards_data = []
    moral_foundations = ["care", "fairness", "liberty", "loyalty", "authority", "sanctity", "none"]
    input_files = {"wizards": "wizards_dialogues.json", "final_argubot": "argubot_final_exp.json",
                   "models_dialogues": "models_dialogues.json"}
    dials_with_scores = {"wizards": {}, "final_argubot": {}, "models_dialogues": {}}

    for key in input_files:
        input_file = input_files[key]
        with open(input_file, "r", encoding="utf-8") as f:
            data = json.load(f)
        for d in data:
            is_wiki = False
            for m in d["messages"]:
                if 'model' in m and (m['model'] == 'wikibot' or m['model'] == 'controlbot'):
                    is_wiki = True
                    break
            if is_wiki:
                continue
            yes_no = 'none'
            k = 'Did you vote for (Leave) or against (Remain) Brexit in the 2016 UK referendum?'
            if k in d['participant_info']:
                if d['participant_info'][k].lower() == 'against (remain)':
                    yes_no = 'no'
                elif d['participant_info'][k].lower() == 'for (leave)':
                    yes_no = 'yes'
                else:
                    yes_no = 'none'

            k = 'In the referendum on whether the UK should remain a member of the EU (BREXIT), how did you vote?'
            if k in d['participant_info']:
                if d['participant_info'][k].lower() == 'remain (against brexit)':
                    yes_no = 'no'
                elif d['participant_info'][k].lower() == 'leave (for brexit)':
                    yes_no = 'yes'
                else:
                    yes_no = 'none'
            k = 'Have you had at least one dose of an approved Covid-19 vaccine?'
            if k in d['participant_info']:
                if d['participant_info'][k].lower() == 'yes':
                    yes_no = 'yes'
                elif d['participant_info'][k].lower() == 'no':
                    yes_no = 'no'
            k = 'Are you a vegan?'
            if k in d['participant_info']:
                if d['participant_info'][k].lower() == 'yes':
                    yes_no = 'yes'
                elif d['participant_info'][k].lower() == 'no':
                    yes_no = 'no'

            if yes_no == 'none':
                continue

            if 'Questions' in d['participant_info']:
                for q in d['participant_info']['Questions']:
                    if "final" in input_file:
                        if label == 'oum':
                            continue
                        if d['participant_info']['Questions'][q]['after'] == -1:
                            continue
                    elif d['participant_info']['Questions'][q]['before'] == -1 or d['participant_info']['Questions'][q]['after'] == -1:
                        continue
                    if 'good reasons' in q.lower():
                        if d['topic'] != 'brexit' and 'not' in q.lower() and yes_no == 'no':
                            continue
                        if d['topic'] != 'brexit' and 'not' not in q.lower() and yes_no == 'yes':
                            continue
                        if 'leave' in q.lower() and yes_no == 'yes':
                            continue
                        if 'remain' in q.lower() and yes_no == 'no':
                            continue
                        if d["_id"] not in dials_with_scores[key]:
                            text = ''
                            dials_with_scores[key][d["_id"]] = {"topic": d["topic"], "dataset": key}
                            for message in d['messages']:
                                if message['role'] == 'admin' or 'modified_argument' not in message:
                                    continue

                                text = text + '\n\n' + '<' + message['role'] + '>' + '\n' + message['modified_argument']
                            dials_with_scores[key][d["_id"]]['text'] = text.strip()
                            final_convs.append(text.strip())

                    if 'good reasons' in q.lower():
                        if False and label == 'oum':
                            final_labels.append(float(d['participant_info']['Questions'][q]['after']) - float(d['participant_info']['Questions'][q]['before']))
                        else:
                            final_labels.append(float(d['participant_info']['Questions'][q]['after']))
                        oum = d['participant_info']['Questions'][q]['after'] - d['participant_info']['Questions'][q]['before'] if "final" not in input_file else None
                        dials_with_scores[key][d["_id"]]["good_reasons"] = {"oum": oum, "after": d['participant_info']['Questions'][q]['after']}
                        if 'before' in d['participant_info']['Questions'][q] and d['participant_info']['Questions'][q]['before'] != -1:
                            dials_with_scores[key][d["_id"]]["good_reasons"]['before'] = d['participant_info']['Questions'][q]['before']
                        else:
                            dials_with_scores[key][d["_id"]]["good_reasons"]['before'] = None

    assert len(final_convs) == len(final_labels)
    return final_convs, final_labels

def get_utterances(text):
    # Splitting the input text into lines
    lines = text.split('\n')
    # Variable to keep the cleaned lines
    cleaned_lines = []
    # Variable to keep track of whether the next line should be added
    add_next_line = False
    for line in lines:
        # If the line is a participant tag, set the flag to add the next line
        if line.strip() == '<participant>':
            add_next_line = True
        elif line.strip() in ['<woz>', '<chatbot>']:
            add_next_line = True
        elif add_next_line:
            # If the flag is set, add this line to the cleaned list and reset the flag
            cleaned_lines.append(line)
            add_next_line = False
    # Join the cleaned lines back into a single string
    cleaned_text = '\n'.join(cleaned_lines)
    return cleaned_text

### Bot/Woz's data
conversations, labels = load_data_oum(label='after')
utterances = [get_utterances(c) for c in conversations]


## 0.2. Wikitac Data

In [2]:
import json
import pandas as pd
import numpy as np
from collections import Counter

def load_data_wikitac():
    with open('../wikitactics.json') as f:
        data = json.load(f)

    conversations = []
    utterances_cleaned = []
    labels = []
    for dispute in data:
        users = list()
        conversation = ''
        utt_cleaned = ''
        for utterance in dispute['utterances']:
            username = utterance['username']
            text = utterance['text']
            conversation += f"<user_id={username}>\n{text}\n\n"
            utt_cleaned += text + '\n\n'
        conversations.append(conversation)
        utterances_cleaned.append(utt_cleaned)
        labels.append(dispute['escalation_label'])

    return conversations, utterances_cleaned, labels

conversations, utterances, labels = load_data_wikitac()

## 0.3. AFD

In [7]:
def load_data_afd():
    # Load the data from the JSON file
    with open('../afd_1000_randomised_dialogues.json', 'r') as json_file:
        data_dict = json.load(json_file)

    # Extract the conversations, utterances, and labels from the data dictionary
    conversations = data_dict['conversations']
    utterances = data_dict['utterances']
    labels = data_dict['labels']

    return conversations, utterances, labels

conversations, utterances, labels = load_data_afd()

# 2. Argument Quality Features

## 2.1. Load **Conversation-level** Data for **Relation Annotation**.




### 2.1.1. Annotating the Argument Quality (AQ).

In [8]:
import collections
import time
import os
import openai
import os
import json

def prompt_gpt4_errorlessly(prompt): 
    ok = False
    while not ok:  # to avoid "ServiceUnavailableError: The server is overloaded or not ready yet."
        try:
            response = openai.ChatCompletion.create(
                        model="gpt-4-1106-preview",
                        messages=[
                                  {"role": "user", "content": prompt}
                                  ],
                        max_tokens = 1000,
                        temperature = 0
                        )
            ok = True

        except Exception as ex:
            print("error", ex)
            print("too much request, sleep for 5 seconds")
            time.sleep(5)

    return response["choices"][0]["message"]["content"]

In [ ]:
conversations, utterances, labels = load_data_afd()

human_args_quality_annotations = []

for i, c in enumerate(conversations):
    # prompt for OUM data
    prompt = f"""{c}\n\n
    The texts above show a dialogue on a controversial topic between an AI chatbot named 'woz' or 'bot' and a human participant. Both entities present arguments on the topic, possibly incorporating various rhetorical strategies, factual accuracy, relevance, coherence, among others. Now, please use chain-of-thought reasoning to evaluate the *average quality of arguments* (i.e., the discussion quality).
    After the Chain-of-Thoughts reasoning step, you should assign an overall quality score to the entire discussion on a scale from 1 to 10, where 1 is of poor quality and 10 is of extremely good quality. Conclude your evaluation with the statement: 'Thus, the quality score of the discussion is: X', where X is the numeric score (real number) you've determined."""

    # prompt for Wikitac
    prompt = f"""{c}\n\n
    The texts above show a dialogue with respect to a potentially controversial edit between two or more individuals from Wikipedia Talk pages. All individuals present arguments on why they think the edit/s is or isn't reasonable, possibly incorporating various rhetorical strategies, factual accuracy, relevance, coherence, among others. Now, please use chain-of-thought reasoning to evaluate the *average quality of arguments* (i.e., the discussion quality).
    After the Chain-of-Thoughts reasoning steps, you should assign an overall quality score to the entire discussion on a scale from 1 to 10, where 1 is of poor quality and 10 is of extremely good quality. Conclude your evaluation with the statement: 'Thus, the quality score of the discussion is: X', where X is the numeric score (real number) you've determined."""

    # prompt for AFD
    prompt = f"""{c}\n\n
    The texts above show a dialogue concerning a proposal for deleting a Wikipedia article, discussed between two or more individuals or users. All individuals present arguments on why they think the proposed deletion is or isn't reasonable, possibly incorporating various rhetorical strategies, factual accuracy, relevance, and coherence, among others. Please use chain-of-thought reasoning to evaluate the *average quality of arguments* (i.e., the discussion quality).
    After the Chain-of-Thoughts reasoning step, you should assign an overall quality score to the entire discussion on a scale from 1 to 10, where 1 is of poor quality and 10 is of extremely good quality. Conclude your evaluation with the statement: 'Thus, the quality score of the discussion is: X', where X is the numeric score (real number) you've determined."""

    
    ok = False
    while not ok: 
        try:
            response_text = prompt_gpt4_errorlessly(prompt)
            ok = True

        except Exception as ex:
            print("error", ex)
            print("too much request, sleep for 5 seconds")
            time.sleep(5)

    print('\n\n\n', i)
    print('\n', response_text)
    human_args_quality_annotations.append(response_text)

import pandas as pd
data = pd.DataFrame({'conversation': conversations, 'label': labels, 'human_AQ_annotation' : human_args_quality_annotations})
# data.to_csv('542_oum_gpt4_AQ_1dim.csv', index=False)
# data.to_csv('213_wikitac_gpt4_AQ_1dim.csv', index=False)
data.to_csv('1000_afd_gpt4_AQ_1dim.csv', index=False)